In [1]:
import torch
import gym
import hydra
from omegaconf import DictConfig
from pytorch_lightning import seed_everything
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hulc.models.decoders.utils.gripper_control import tcp_to_world_frame, world_to_tcp_frame
import numpy as np

In [3]:
def skill_type_classifier(acts):
    energy = 0.
    energy_g = 0.
    _, T, _ = acts.shape
    for i in range(T):
        energy += abs(acts[:, i, :6])
    for i in range(T-1):
        energy_g += abs(acts[:, i+1, 6]- acts[:, i, 6])
    translation = (energy[:, 0] + energy[:, 1] + energy[:, 2]) / 3
    rotation = (energy[:, 3] + energy[:, 4] + energy[:, 5]) / 3
    gripper = energy_g
    return translation, rotation, gripper

In [4]:
with initialize(version_base=None, config_path="../../conf_sg"):
    cfg = compose(config_name="config.yaml", overrides=['datamodule.root_data_dir=dataset/calvin_debug_dataset'])
    datamodule = hydra.utils.instantiate(cfg.datamodule)
    print(datamodule.training_dir)
    datamodule.prepare_data()
    datamodule.setup()
    train_dataloader = datamodule.train_dataloader()
    a_t = []
    a_r = []
    a_g = []
    for data in train_dataloader:
        gt_actions = data['actions']
        robot_obs = data['state_info']['robot_obs']
        tcp_actions = world_to_tcp_frame(gt_actions, robot_obs)
        t, r, g = skill_type_classifier(tcp_actions)
        a_t += t
        a_r += r
        a_g += g

    a_t = np.array(a_t)
    a_r = np.array(a_r)
    a_g = np.array(a_g)
    print(a_t.mean(), a_t.std())
    print(a_r.mean(), a_r.std())
    print(a_g.mean(), a_g.std())

/Users/Garen/opt/anaconda3/envs/hulc/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


/Users/Garen/hulc/dataset/calvin_debug_dataset/training


/Users/Garen/opt/anaconda3/envs/hulc/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:133: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/Users/Garen/opt/anaconda3/envs/hulc/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


KeyboardInterrupt: 